# Use this notebook to make plots with wrfout files
## Notebook compiled by Jeremiah Otero
## Adapted from Steve Nesbitt's wrf_plot_FULL_domain.py
Link: https://github.com/swnesbitt/wrfplotting/blob/master/wrf_plot_FULL_domain.py

In [1]:
## Bring in the important stuff!

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys, os
import xarray as xr

from netCDF4 import Dataset
from matplotlib.cm import get_cmap
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
from pandas import Timestamp
from wrf import to_np, getvar, smooth2d, get_basemap, latlon_coords, extract_times, ALL_TIMES, interplevel
from glob import glob

mpl.use('Agg')
%matplotlib inline

/glade/u/apps/ch/opt/pythonpkgs/2.7/xarray/0.9.5/intel/16.0.3/lib/python2.7/site-packages/xarray/core/formatting.py:16: FutureWarning: The pandas.tslib module is deprecated and will be removed in a future version.
  from pandas.tslib import OutOfBoundsDatetime
/glade/u/apps/ch/opt/pythonpkgs/2.7/matplotlib/2.0.2/gnu/6.3.0/lib/python2.7/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
## Make fonts look nice

mpl.font_manager._rebuild()

mpl.rcParams['font.family'] = 'Arial'
mpl.rcParams['font.style'] = 'normal'

In [3]:
## Define some important landmarks

def landmarks():
    landmark_dict = {'C':(-64.2123,-31.3154),
                    'M':(-68.7987,-32.8278),
                    #'3':(-64.1131,-32.1767),
                    #'4':(-64.34992,-33.13067), 
                    #'Y':(-64.7545,-32.1062),
                    #'S':(-63.978435,-31.715689),
                    'SR':(-68.285522,-34.587997),
                    'SL':(-66.33694,-33.30278)}
    return landmark_dict

landmark_dict=landmarks()

In [4]:
## Select zoom
## available options are: 'full' , 'cordoba_zoom' , 'mendoza_zoom'

zoom='full'

In [5]:
## Bring in wrfout data as 'filenames'

## wrfout directories: named based on ensemble physical parameterizations - edits to namelist.input
    ## mp_physics: 8 = Thompson; 10 = Morrison; 17 = NSSL
    ## bl_pbl_physics: 1 = YSU; 2 = MYJ

path = '/glade/work/jpiers/WRFV3/run/wrfout_10_2' # set path to where wrfout files are located
#filenames = sorted(glob(path+"wrfout_d0*")) # define filenames, sorted by time
filenames = sorted(glob('/glade/work/jpiers/WRFV3/run/wrfout_10_2/wrfout_d0*')) # define filenames, sorted by time

titlestr='GFS-WRF' # create title for figure purposes
modname='GFS' # identify model for figure purposes

In [6]:
## Create lists 'files' and 'times' that will be used for making plots
## Print times - for diagnostic purposes

files=[]
times=[]

for file in filenames:
        files.append(Dataset(file))
        print(os.path.basename(file)[11:])
        times.append(pd.to_datetime(os.path.basename(file)[11:],format='%Y-%m-%d_%H:%M:%S'))

2016-12-25_00:00:00
2016-12-25_01:00:00
2016-12-25_02:00:00
2016-12-25_03:00:00
2016-12-25_04:00:00
2016-12-25_05:00:00
2016-12-25_06:00:00
2016-12-25_07:00:00
2016-12-25_08:00:00
2016-12-25_09:00:00
2016-12-25_10:00:00
2016-12-25_11:00:00
2016-12-25_12:00:00
2016-12-25_13:00:00
2016-12-25_14:00:00
2016-12-25_15:00:00
2016-12-25_16:00:00
2016-12-25_17:00:00
2016-12-25_18:00:00
2016-12-25_19:00:00
2016-12-25_20:00:00
2016-12-25_21:00:00
2016-12-25_22:00:00
2016-12-25_23:00:00
2016-12-26_00:00:00
2016-12-26_01:00:00
2016-12-26_02:00:00
2016-12-26_03:00:00


IOError: NetCDF: Unknown file format

In [ ]:
ls /glade2/work/jpiers/WRFV3/run/wrfout_10_2

In [ ]:
## Define function 'make_plot'

def make_plot(cffield,lfield,lfield2,ufld,vfld,params):
# Get the latitude and longitude points

    print(params['time_index'])
    
    lats, lons = latlon_coords(cffield)

    # Get the basemap object

    if params['zoom'] == 'full':
        bm = Basemap(projection='lcc',width=3000*550,height=3000*375,
        resolution='i',lat_1=-32.8,lat_2=-32.8,lat_0=-32.8,lon_0=-67.0)
        fs=12
        params['skip']=17

    if params['zoom'] == 'cordoba_zoom':
        bm = Basemap(projection='lcc',width=1500*550,height=1500*375,
        resolution='i',lat_1=-32.2,lat_2=-32.2,lat_0=-32.2,lon_0=-65.0)
        fs=14 
        params['skip']=9
    
    if params['zoom'] == 'mendoza_zoom':
        bm = Basemap(projection='lcc',width=1500*550,height=1500*375,
        resolution='i',lat_1=-33.2,lat_2=-33.2,lat_0=-33.2,lon_0=-69.0)
        fs=14 
        params['skip']=9

    # Create a figure
    fig = plt.figure(figsize=(12,9))

    # Add geographic outlines
    bm.drawcoastlines(linewidth=0.75)
    bm.drawstates(linewidth=1.)
    bm.drawcountries(linewidth=1.)

    # Convert the lats and lons to x and y.  Make sure you convert the lats and lons to
    # numpy arrays via to_np, or basemap crashes with an undefined RuntimeError.
    x, y = bm(to_np(lons), to_np(lats))


    if lfield is not None:
        CS=bm.contour(x, y, to_np(lfield), 10, colors="black", levels=params['llevels'],linewidths=1.0)
        plt.clabel(CS, inline=1, fontsize=12, fmt='%d')

    if lfield2 is not None:
        CS=bm.contour(x, y, to_np(lfield2), 10, colors="dimgrey", levels=params['llevels2'],linewidths=2.25)
        #plt.clabel(CS, inline=1, fontsize=12, fmt='%d')
    
    if ufld is not None:
        bm.barbs(x[::params['skip'],::params['skip']], 
                 y[::params['skip'],::params['skip']], 
                 to_np(ufld[::params['skip'],::params['skip']]),
                 to_np(vfld[::params['skip'],::params['skip']]), length=5, linewidth=0.75, zorder=10)

    if not('lalpha' in params):
        params['lalpha']=None
        

    # Draw the contours and filled contours
    bm.contourf(x, y, to_np(cffield), 10, cmap=get_cmap(params['ccmap']), levels=params['clevels'], extend='both',
               alpha=params['lalpha'])


    parallels = np.arange(-50.,-10.,2.)
    # labels = [left,right,top,bottom]
    bm.drawparallels(parallels,labels=[False,True,False,False],linewidth=0.5,dashes=[2,2])
    meridians = np.arange(-90.,-50.,2.)
    bm.drawmeridians(meridians,labels=[False,False,False,True],linewidth=0.5,dashes=[2,2])

    # Add a color bar
    plt.colorbar(shrink=.62, extend='both')

    timediff=params['times'][params['time_index']]-params['times'][0]
    timediff_secs=int(timediff.total_seconds()//3600)

    plt.title(titlestr+' '+cffield.description+' ('+cffield.units+')\n'+
             "Initialized: "+params['times'][0].strftime('%Y-%m-%d %H:%M')+"Z Forecast hour: "+'{:03d}'.format(timediff_secs)+" Valid: "+params['times'][params['time_index']].strftime('%Y-%m-%d %H:%M')+'Z')

    for key in landmark_dict.keys():
        kx,ky=bm(landmark_dict[key][0],landmark_dict[key][1])
        plt.text(kx,ky,key,fontsize=fs,
                        ha='center',va='center',color='b')
    #fig.figimage(im, fig.bbox.xmax-290, fig.bbox.ymin,zorder=10)


## Make and save plots!
See Nesbitt's code, as named above, to bring in the params for different variables. Some modifications necessary, as outlined below.

In [ ]:
#=====================PRECIPITABLE WATER=========================

## Important: the following through tindex=i must be added to Nesbitt's code for each variable, such as pw.
## Indentation matters!

outpath = 'DEC16_images/precip_water' # change directory name to fit variable
for i,f in enumerate(files):
    if i>2:                           # change 'i>2' depending on how many indices aka times you want iterated
        break
    tindex=i
    
    params={'outpath':outpath,
            'modname':modname,
            'modfld':'Precipitable_Water',
            'cfield':'pw',
            'clevels':np.arange(15,75,5),
            'ccmap':"viridis_r",
            'llevels':None,
            'llevels2':[1000],
            'time_index':tindex,
            'times':times,
            'zoom':zoom,
            'skip':17}

    cffield = getvar(files, params['cfield'], timeidx=params['time_index'])
    lfield = None
    lfield2 = getvar(files, 'ter', timeidx=params['time_index'], units='m')
    ufld = None
    vfld = None

    make_plot(cffield,lfield,lfield2,ufld,vfld,params)
    

## Important: add the following to create proper directory and save .png in it
    
    ## Make directory for variable as defined above
    os.system('mkdir -p '+outpath)

    # save figure in outpath. named based on variable and time. must change '/pw_' for other variables
    plt.savefig(outpath+'/pw_'+times[i].strftime('%Y-%m-%d_%H:%M:%S')+'.png', bbox_inches="tight", dpi=200)


In [ ]:
#=========================1 km DBZ=========================

## Important: the following through tindex=i must be added to Nesbitt's code for each variable, such as pw.
## Indentation matters!

outpath = 'DEC16_images/dbz_1km' # change directory name to fit variable
for i,f in enumerate(files):
    if i>61:                           # change 'i>2' depending on how many indices aka times you want iterated
        break
    tindex=i

    params={'outpath':outpath,
            'modname':modname,
            'modfld':'1km_Radar_Reflectivity',
            'cfield':'radar_1km',
            'clevels':np.arange(5,75,5.),
            'ccmap':"gist_ncar",
            'llevels':np.arange(510,606,6),
            'llevels2':[1000],
            'time_index':tindex,
            'times':times,
            'zoom':zoom,
            'skip':17}

    dbz = getvar(files, 'dbz', timeidx=params['time_index'])
    ter = getvar(files, 'ter', timeidx=params['time_index'], units="m")
    z = getvar(files, 'z', timeidx=params['time_index'], units="m")

    ter_3d=np.tile(ter.values,[41,1,1])
    z.values=z.values-ter.values
    cffield = interplevel(dbz, z, 1000)
    cffield.values=np.ma.masked_less(cffield.values,5.)
    cffield.attrs['description']='1 km AGL radar reflectivity'
    lfield = None
    lfield2 = getvar(files, 'ter', timeidx=params['time_index'], units='m')
    ufld=None
    vfld=None

    make_plot(cffield,lfield,lfield2,ufld,vfld,params)

## Important: add the following to create proper directory and save .png in it
    
    ## Make directory for variable as defined above
    os.system('mkdir -p '+outpath)

    # save figure in outpath. named based on variable and time. must change '/pw_' for other variables
    plt.savefig(outpath+'/dbz_1km_'+times[i].strftime('%Y-%m-%d_%H:%M:%S')+'.png', bbox_inches="tight", dpi=200)

In [ ]:
#====================TOTAL PRECIPITAITON=========================

## Important: the following through tindex=i must be added to Nesbitt's code for each variable, such as pw.
## Indentation matters!

outpath = 'DEC16_images/total_precip' # change directory name to fit variable
for i,f in enumerate(files):
    if i>2:                           # change 'i>2' depending on how many indices aka times you want iterated
        break
    tindex=i
    
    params={'outpath':outpath,
            'modname':modname,
            'modfld':'Total_Precipitation',
            'cfield':'RAINNC',
            'clevels':np.arange(0,165,10),
            'ccmap':"ocean_r",
            'llevels':None,
            'llevels2':[1000],
            'time_index':tindex,
            'times':times,
            'zoom':zoom,
            'skip':17}

    cffield=None
    cffield = getvar(files, params['cfield'], timeidx=params['time_index'])
    cffield.attrs['description']='total precipitation'
    cffield.attrs['units']='mm'
    lfield = None
    lfield2 = getvar(files, 'ter', timeidx=params['time_index'], units='m')
    ufld = None
    vfld = None

    make_plot(cffield,lfield,lfield2,ufld,vfld,params)

## Important: add the following to create proper directory and save .png in it
    
    ## Make directory for variable as defined above
    os.system('mkdir -p '+outpath)

    # save figure in outpath. named based on variable and time. must change '/pw_' for other variables
    plt.savefig(outpath+'/totprec_'+times[i].strftime('%Y-%m-%d_%H:%M:%S')+'.png', bbox_inches="tight", dpi=200)

In [ ]:
#=====================2 M TEMP=========================
    
## Important: the following through tindex=i must be added to Nesbitt's code for each variable, such as pw.
## Indentation matters!

outpath = 'DEC16_images/T2' # change directory name to fit variable
for i,f in enumerate(files):
    if i>2:                           # change 'i>2' depending on how many indices aka times you want iterated
        break
    tindex=i
    
    params={'outpath':outpath,
            'modname':modname,
            'modfld':'2m_Temperature',
            'cfield':'T2',
            'clevels':np.arange(-15,40,2.5),
            'ccmap':"plasma",
            'llevels':np.arange(970,1040,4),
            'llevels2':[1000],
            'time_index':tindex,
            'times':times,
            'zoom':zoom,
            'skip':17}

    cffield = getvar(files, params['cfield'], timeidx=params['time_index'],units='degC')
    cffield.attrs['description']='2 m temperature'
    cffield.attrs['temperature']='degC'
    cffield.values=cffield.values-273.15
    lfield = getvar(files, 'slp', timeidx=params['time_index'], units='hPa')
    lfield2 = getvar(files, 'ter', timeidx=params['time_index'], units='m')
    uvmet = getvar(files, 'uvmet10', timeidx=params['time_index'], units='kt')
    ufld = uvmet.isel(u_v=0)
    vfld = uvmet.isel(u_v=1)

    make_plot(cffield,lfield,lfield2,ufld,vfld,params)
    
## Important: add the following to create proper directory and save .png in it
    
    ## Make directory for variable as defined above
    os.system('mkdir -p '+outpath)

    # save figure in outpath. named based on variable and time. must change '/pw_' for other variables
    plt.savefig(outpath+'/T2_'+times[i].strftime('%Y-%m-%d_%H:%M:%S')+'.png', bbox_inches="tight", dpi=200)